# Μηχανή Αναζήτησης - Συγκεντρωμένα Προγράμματα

Στο Notebook αυτό θα βρεις **όλα τα προγράμματα** που απαρτίζουν τη μηχανή αναζήτησης:

1. **Wikipedia Crawler** (Συλλογή δεδομένων)
2. **Preprocessing Script** (Προεπεξεργασία κειμένου)
3. **create_inverted_index** (Δημιουργία απλού Inverted Index)
4. **create_inverted_index_with_tfidf** (Δημιουργία Inverted Index με TF-IDF)
5. **Boolean Search Engine** (Χρησιμοποιεί απλό inverted index)
6. **Enhanced Search Engine** (TF-IDF, VSM, BM25)
7. **Evaluation** (Υπολογισμός μετρικών ακρίβειας, ανάκλησης, F1)


## 1) Wikipedia Crawler

**Σκοπός**: Συλλέγει δεδομένα από το Wikipedia, αντλώντας τον τίτλο και την πρώτη παράγραφο από κάθε σελίδα. Στη συνέχεια, τα αποθηκεύει σε μορφή JSON (π.χ. `wiki_definitions.json`).

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def fetch_html(url):
    """
    Στέλνει αίτημα GET σε ένα URL και επιστρέφει το HTML της σελίδας.
    Επιστρέφει None αν υπάρξει σφάλμα δικτύου ή άλλο HTTP σφάλμα.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Εάν το response != 200, σηκώνεται exception
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_first_paragraph(html):
    """
    Αναλύει το HTML με BeautifulSoup και επιστρέφει τον τίτλο και την πρώτη
    παράγραφο που βρει.
    """
    soup = BeautifulSoup(html, 'html.parser')

    # Αφαιρούμε tags <sup> (συνήθως citations)
    for sup_tag in soup.find_all('sup'):
        sup_tag.decompose()

    # Αφαιρούμε tags <span class="reference"> (επίσης citations)
    for ref_tag in soup.find_all('span', class_='reference'):
        ref_tag.decompose()

    # Βρίσκουμε τον τίτλο από το <title>
    title_tag = soup.find('title')
    title = title_tag.text if title_tag else "No Title"

    # Παίρνουμε την πρώτη παράγραφο
    paragraphs = soup.find_all('p')
    first_paragraph = ""
    for p in paragraphs:
        text = p.get_text(separator=" ", strip=True)
        if text:
            first_paragraph = text
            break

    return title, first_paragraph

def collect_wikipedia_definitions(url_list, output_filename="wiki_definitions.json"):
    """
    Δέχεται μια λίστα από Wikipedia URLs και για κάθε σελίδα:
      - Ανακτά HTML,
      - Εξάγει τον τίτλο και την πρώτη παράγραφο,
      - Αποθηκεύει σε JSON.
    """
    collected_data = []

    for url in url_list:
        print(f"Fetching: {url}")
        html = fetch_html(url)
        if html:
            title, first_paragraph = extract_first_paragraph(html)
            data = {
                'url': url,
                'title': title,
                'definition': first_paragraph
            }
            collected_data.append(data)
        else:
            data = {
                'url': url,
                'title': "Error",
                'definition': "Could not fetch page"
            }
            collected_data.append(data)

    # Τέλος, αποθήκευση σε αρχείο JSON
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(collected_data, f, ensure_ascii=False, indent=4)

    print(f"\nTotal pages fetched: {len(collected_data)}")
    print(f"Data saved to '{output_filename}'")

if __name__ == "__main__":
    wikipedia_pages = [
        "https://en.wikipedia.org/wiki/Web_crawler",
        "https://en.wikipedia.org/wiki/Python_(programming_language)",
        # ... (βάλε εδώ όλα τα URLs που έχεις)
    ]

    collect_wikipedia_definitions(
        url_list=wikipedia_pages,
        output_filename="wiki_definitions.json"
    )


### Επεξήγηση Συναρτήσεων (Wikipedia Crawler)

- **`fetch_html(url)`**:
  - Στέλνει αίτημα GET στο `url`.
  - Κάνει `response.raise_for_status()` για να πιάσει σφάλματα HTTP.
  - Επιστρέφει το `response.text` αν όλα πάνε καλά.

- **`extract_first_paragraph(html)`**:
  - Χρησιμοποιεί `BeautifulSoup` για ανάλυση HTML.
  - Αφαιρεί citations (`<sup>`, `<span class="reference">`).
  - Βρίσκει τον τίτλο με `soup.find('title')`.
  - Παίρνει την πρώτη **μη κενή** παράγραφο.

- **`collect_wikipedia_definitions(url_list, output_filename)`**:
  - Για κάθε URL, καλεί `fetch_html` και `extract_first_paragraph`.
  - Δημιουργεί μια λίστα `collected_data` και την αποθηκεύει σε `wiki_definitions.json`.


## 2) Προεπεξεργασία Κειμένου (Preprocessing)

**Σκοπός**: Παίρνει το `wiki_definitions.json` και καθαρίζει τα κείμενα. Κάνει μετατροπή σε πεζά, αφαίρεση ειδικών χαρακτήρων, stopwords, stemming, και αποθηκεύει την επεξεργασμένη μορφή σε `wiki_definitions_cleaned.json`.

In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess_text(text):
    """
    - Μετατρέπει σε lowercase
    - Αφαιρεί ειδικούς χαρακτήρες
    - Χωρίζει το κείμενο σε tokens
    - Αφαιρεί stopwords
    - Εφαρμόζει stemming
    """
    # 1. Lowercase
    text = text.lower()

    # 2. Αφαιρούμε ειδικούς χαρακτήρες διατηρώντας μόνο γράμματα/αριθμούς
    text = re.sub(r'[^a-z0-9\s]', ' ', text)

    # 3. Tokenization (απλό split)
    tokens = text.split()

    # 4. Αφαίρεση stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # 5. Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    return stemmed_tokens

def main(input_json, output_json):
    """
    1. Διαβάζει το input_json
    2. Καλεί preprocess_text για κάθε 'definition'
    3. Αποθηκεύει σε output_json
    """
    with open(input_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    cleaned_data = []
    for item in data:
        original_definition = item.get('definition', '')
        cleaned_tokens = preprocess_text(original_definition)

        new_item = {
            'url': item.get('url', ''),
            'title': item.get('title', ''),
            'original_definition': original_definition,
            'cleaned_definition': cleaned_tokens
        }
        cleaned_data.append(new_item)

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

    print(f"Preprocessing complete. Saved to {output_json}")

if __name__ == "__main__":
    input_file = "wiki_definitions.json"
    output_file = "wiki_definitions_cleaned.json"
    main(input_file, output_file)


### Επεξήγηση Συναρτήσεων (Προεπεξεργασία)

- **`preprocess_text(text)`**:
  - Χρησιμοποιεί **regular expressions** για να αφαιρέσει ειδικούς χαρακτήρες.
  - Κάνει **tokenization** με `split()`.
  - Αφαιρεί **stopwords** μέσω `stopwords.words('english')`.
  - Εφαρμόζει **PorterStemmer()** για να πάρει τις ρίζες των λέξεων.

- **`main(input_json, output_json)`**:
  - Διαβάζει το αρχείο `wiki_definitions.json`.
  - Για κάθε εγγραφή, προεπεξεργάζεται το `definition`.
  - Αποθηκεύει το αποτέλεσμα στο `wiki_definitions_cleaned.json`.

## 3) Δημιουργία Inverted Index

**Σκοπός**: Διαβάζει τα "καθαρισμένα" κείμενα από το `wiki_definitions_cleaned.json` και φτιάχνει ένα απλό inverted index στη μορφή `λέξη -> [doc_ids]`.

In [ ]:
import json
from collections import defaultdict

def create_inverted_index(input_json, output_json):
    """
    1. Διαβάζει τα καθαρισμένα δεδομένα (wiki_definitions_cleaned.json)
    2. Δημιουργεί ανεστραμμένο ευρετήριο (inverted index)
    3. Αποθηκεύει σε output_json
    """
    with open(input_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    inverted_index = defaultdict(list)

    for i, doc in enumerate(data):
        doc_id = i + 1  # ξεκινάμε από 1
        tokens = doc.get('cleaned_definition', [])
        for token in tokens:
            if doc_id not in inverted_index[token]:
                inverted_index[token].append(doc_id)

    # Μετατρέπουμε σε κανονικό dict
    inverted_index = dict(inverted_index)

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(inverted_index, f, ensure_ascii=False, indent=4)

    print(f"Inverted index created and saved to {output_json}")

if __name__ == "__main__":
    input_file = "wiki_definitions_cleaned.json"
    output_file = "inverted_index.json"
    create_inverted_index(input_file, output_file)


### Επεξήγηση (create_inverted_index)

- **`create_inverted_index`**:
  - Χρησιμοποιεί `defaultdict(list)` για να προσθέτει doc_ids σε λίστες.
  - Κάθε token αντιστοιχίζεται σε όλα τα `doc_id` όπου εμφανίζεται.
  - Αποθηκεύει ένα dict του τύπου:
    ```json
    {
      "machine": [1, 5, 12],
      "learn": [1, 8]
      ...
    }
    ```


## 4) Inverted Index με TF-IDF

**Σκοπός**: Επιπλέον αποθηκεύουμε τη συχνότητα εμφάνισης μιας λέξης σε ένα doc (TF) και την τιμή **TF-IDF**.


In [ ]:
import json
from collections import defaultdict
import math

def create_inverted_index_with_tfidf(input_json, output_json):
    with open(input_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    total_docs = len(data)

    freq_index = defaultdict(lambda: defaultdict(int))

    # Υπολογίζουμε πόσες φορές εμφανίζεται μια λέξη σε κάθε doc (TF)
    for i, doc in enumerate(data):
        doc_id = i + 1
        tokens = doc.get('cleaned_definition', [])
        for token in tokens:
            freq_index[token][doc_id] += 1

    # Υπολογισμός IDF
    idf_scores = {}
    for term, docs_dict in freq_index.items():
        df = len(docs_dict)  # πόσα docs έχουν αυτό το term
        idf_scores[term] = math.log(total_docs / (1 + df))

    # Δημιουργία inverted index με tf & tf-idf
    inverted_index = {}
    for term, docs_dict in freq_index.items():
        inverted_index[term] = {}
        for doc_id, tf in docs_dict.items():
            tf_idf = tf * idf_scores[term]
            inverted_index[term][doc_id] = {
                "tf": tf,
                "tf-idf": tf_idf
            }

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(inverted_index, f, ensure_ascii=False, indent=4)
    print(f"Inverted index with TF and TF-IDF saved to {output_json}")

if __name__ == "__main__":
    input_file = "wiki_definitions_cleaned.json"
    output_file = "inverted_index_tfidf.json"
    create_inverted_index_with_tfidf(input_file, output_file)


### Επεξήγηση (create_inverted_index_with_tfidf)

- **TF** (Term Frequency): Για κάθε `(term, doc_id)` μετράμε πόσες φορές εμφανίζεται η λέξη στο doc.
- **IDF** (Inverse Document Frequency): `idf = log(total_docs / (1 + df))`, όπου `df` είναι πόσα docs περιέχουν τη λέξη.
- **tf-idf** = TF * IDF.
- Το τελικό inverted index αποθηκεύεται σε μορφή:
  ```json
  {
    "machine": {
      1: {"tf": 3, "tf-idf": 3.5},
      5: {"tf": 1, "tf-idf": 1.2},
      ...
    },
    ...
  }
  ```

## 5) Απλή Μηχανή Αναζήτησης (Boolean)

**Σκοπός**: Χρησιμοποιεί το `inverted_index.json` (χωρίς TF-IDF). Δίνει τη δυνατότητα Boolean αναζητήσεων (AND, OR, NOT) ή απλών (default OR).

In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def load_inverted_index(index_file):
    try:
        with open(index_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Could not find the file {index_file}.")
        return None

def load_documents(docs_file):
    """
    Φορτώνει τη λίστα των εγγράφων (JSON)
    """
    try:
        with open(docs_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Could not find the file {docs_file}.")
        return None

def preprocess_query(query):
    query = query.lower()
    query = re.sub(r'[^a-z0-9\s]', ' ', query)
    tokens = query.split()
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

def process_boolean_query(tokens, inverted_index):
    result_set = None
    operation = None
    for token in tokens:
        if token in ["and", "or", "not"]:
            operation = token
        else:
            docs_for_token = set(map(int, inverted_index.get(token, [])))
            if result_set is None:
                result_set = docs_for_token
            else:
                if operation == "and":
                    result_set &= docs_for_token
                elif operation == "or":
                    result_set |= docs_for_token
                elif operation == "not":
                    result_set -= docs_for_token
                else:
                    result_set |= docs_for_token
    if result_set is None:
        result_set = set()
    return result_set

def process_simple_query(tokens, inverted_index, default_operator="or"):
    result_set = set()
    for i, token in enumerate(tokens):
        docs_for_token = set(map(int, inverted_index.get(token, [])))
        if i == 0:
            result_set = docs_for_token
        else:
            if default_operator == "or":
                result_set |= docs_for_token
            elif default_operator == "and":
                result_set &= docs_for_token
    return result_set

def process_query(query, inverted_index, default_operator="or"):
    tokens = query.lower().split()
    has_boolean_ops = any(tok in ["and", "or", "not"] for tok in tokens)
    if has_boolean_ops:
        stemmed_tokens = preprocess_query(query)
        return process_boolean_query(stemmed_tokens, inverted_index)
    else:
        stemmed_tokens = preprocess_query(query)
        return process_simple_query(stemmed_tokens, inverted_index, default_operator)

def main():
    index_file = "inverted_index.json"
    docs_file = "wiki_definitions_cleaned.json"

    inverted_index = load_inverted_index(index_file)
    if not inverted_index:
        return
    documents = load_documents(docs_file)
    if not documents:
        return

    print("Welcome to the Search Engine!")
    print("(Boolean operators: AND, OR, NOT) or just words.")
    print("Type 'exit' to quit.\n")

    while True:
        query = input("Enter your query: ")
        if query.lower() == "exit":
            print("Exiting...")
            break

        result_docs = process_query(query, inverted_index)
        if result_docs:
            print(f"\nFound {len(result_docs)} documents:")
            for doc_id in sorted(result_docs):
                real_index = doc_id - 1
                if 0 <= real_index < len(documents):
                    doc = documents[real_index]
                    title = doc.get('title', 'No Title')
                    original_def = doc.get('original_definition', 'No Definition')
                    snippet = original_def[:200] + "..." if len(original_def) > 200 else original_def
                    print(f"  Document ID: {doc_id}")
                    print(f"    Title: {title}")
                    print(f"    Definition: {snippet}\n")
                else:
                    print(f"  Document ID: {doc_id} (not found)\n")
        else:
            print("No results found.")

if __name__ == "__main__":
    main()


### Επεξήγηση (Boolean Search)

- **`process_boolean_query(tokens, inverted_index)`**:
  - Διαβάζει διαδοχικά τους tokens.
  - Αν συναντήσει λογικό τελεστή (`AND`, `OR`, `NOT`), ενημερώνει την τρέχουσα πράξη.
  - Αν συναντήσει λέξη, βρίσκει τα doc_ids από το `inverted_index`.
  - Εφαρμόζει την τρέχουσα πράξη στο `result_set`.

- **`process_simple_query(tokens, ...)`**:
  - Υλοποιεί OR/AND σε όλα τα tokens αν δεν υπάρχει ρητός τελεστής.

- **`process_query(query, inverted_index, default_operator)`**:
  - Προεπεξεργάζεται το query με stemming κ.λπ.
  - Επιλέγει αν θα εκτελέσει Boolean ή απλή αναζήτηση.

- **Εκτέλεση**:
  - Φορτώνει `inverted_index.json`, `wiki_definitions_cleaned.json`.
  - Δέχεται ερώτημα χρήστη.
  - Εμφανίζει doc_ids και αποσπάσματα.

## 6) Εμπλουτισμένη Μηχανή Αναζήτησης (TF-IDF, VSM, BM25)

**Σκοπός**: Φορτώνει το `inverted_index_tfidf.json`, δίνει δυνατότητα **αξιοποίησης TF-IDF**, **Vector Space Model** (cosine similarity), και **Okapi BM25**.

In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

def load_inverted_index(index_file):
    try:
        with open(index_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Could not find the file {index_file}.")
        return None

def load_documents(docs_file):
    try:
        with open(docs_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Could not find the file {docs_file}.")
        return None

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

def preprocess_documents(documents):
    processed_texts = []
    for doc in documents:
        original_def = doc.get('original_definition', '')
        tokens = preprocess_text(original_def)
        processed_text = ' '.join(tokens)
        processed_texts.append(processed_text)
    return processed_texts

def initialize_vsm(processed_texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)
    return vectorizer, tfidf_matrix

def initialize_bm25(processed_texts):
    tokenized_texts = [text.split() for text in processed_texts]
    bm25 = BM25Okapi(tokenized_texts)
    return bm25

# boolean ή απλή αναζήτηση
def process_boolean_query(tokens, inverted_index):
    result_set = None
    operation = None
    for token in tokens:
        if token in ["and", "or", "not"]:
            operation = token
        else:
            docs_for_token = {int(doc) for doc in inverted_index.get(token, [])}
            if result_set is None:
                result_set = docs_for_token
            else:
                if operation == "and":
                    result_set &= docs_for_token
                elif operation == "or":
                    result_set |= docs_for_token
                elif operation == "not":
                    result_set -= docs_for_token
                else:
                    result_set |= docs_for_token
    return result_set if result_set is not None else set()

def process_simple_query(tokens, inverted_index, default_operator="or"):
    result_set = set()
    for i, token in enumerate(tokens):
        docs_for_token = {int(doc) for doc in inverted_index.get(token, [])}
        if i == 0:
            result_set = docs_for_token
        else:
            if default_operator == "or":
                result_set |= docs_for_token
            elif default_operator == "and":
                result_set &= docs_for_token
    return result_set

def calculate_ranking_boolean(result_docs, inverted_index, query_terms, vectorizer, tfidf_matrix):
    doc_scores = {}
    for term in query_terms:
        if term in vectorizer.vocabulary_:
            term_index = vectorizer.vocabulary_[term]
            term_tfidf = tfidf_matrix[:, term_index].toarray().flatten()
            for doc_id in result_docs:
                doc_id = int(doc_id)
                score = term_tfidf[doc_id - 1]
                doc_scores[doc_id] = doc_scores.get(doc_id, 0) + score
    return doc_scores

def calculate_ranking_vsm(query, vectorizer, tfidf_matrix):
    query_vector = vectorizer.transform([query])
    return cosine_similarity(query_vector, tfidf_matrix).flatten()

def calculate_ranking_bm25(query, bm25):
    tokens = query.split()
    return bm25.get_scores(tokens)

def process_query(
    query, inverted_index, vectorizer, tfidf_matrix, bm25, documents,
    default_operator="or"
):
    tokens = query.lower().split()
    has_boolean_ops = any(tok in ["and", "or", "not"] for tok in tokens)

    if has_boolean_ops:
        stemmed_tokens = preprocess_text(query)
        result_docs = process_boolean_query(stemmed_tokens, inverted_index)
        if result_docs:
            query_terms = [tok for tok in stemmed_tokens if tok not in ["and", "or", "not"]]
            doc_scores = calculate_ranking_boolean(
                result_docs, inverted_index, query_terms, vectorizer, tfidf_matrix
            )
            ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        else:
            ranked_docs = []
    else:
        stemmed_tokens = preprocess_text(query)
        result_docs = process_simple_query(stemmed_tokens, inverted_index, default_operator)
        if result_docs:
            print("\nSelect Ranking Algorithm:")
            print("1. TF-IDF Sum")
            print("2. Vector Space Model (Cosine Similarity)")
            print("3. Okapi BM25")
            choice = input("Enter the number of the ranking algorithm (1-3): ")

            if choice == "1":
                query_terms = [tok for tok in stemmed_tokens]
                doc_scores = calculate_ranking_boolean(
                    result_docs, inverted_index, query_terms,
                    vectorizer, tfidf_matrix
                )
                ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
            elif choice == "2":
                original_query = ' '.join(stemmed_tokens)
                cosine_similarities = calculate_ranking_vsm(original_query, vectorizer, tfidf_matrix)
                ranked_docs = sorted([
                    (doc_id, cosine_similarities[doc_id - 1])
                    for doc_id in result_docs
                ], key=lambda x: x[1], reverse=True)
            elif choice == "3":
                original_query = ' '.join(stemmed_tokens)
                bm25_scores = calculate_ranking_bm25(original_query, bm25)
                ranked_docs = sorted([
                    (doc_id, bm25_scores[doc_id - 1]) for doc_id in result_docs
                ], key=lambda x: x[1], reverse=True)
            else:
                print("Invalid choice. Defaulting to TF-IDF Sum.")
                query_terms = [tok for tok in stemmed_tokens]
                doc_scores = calculate_ranking_boolean(
                    result_docs, inverted_index, query_terms,
                    vectorizer, tfidf_matrix
                )
                ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        else:
            ranked_docs = []

    return ranked_docs

def main():
    index_file = "inverted_index_tfidf.json"
    docs_file = "wiki_definitions_cleaned.json"

    inverted_index = load_inverted_index(index_file)
    if not inverted_index:
        return
    documents = load_documents(docs_file)
    if not documents:
        return

    print("Preprocessing documents for VSM and BM25...")
    processed_texts = preprocess_documents(documents)

    print("Initializing Vector Space Model (VSM)...")
    vectorizer, tfidf_matrix = initialize_vsm(processed_texts)

    print("Initializing Okapi BM25...")
    bm25 = initialize_bm25(processed_texts)

    print("\nWelcome to the Enhanced Search Engine!")
    print("Type 'exit' to quit.\n")

    while True:
        query = input("Enter your query: ")
        if query.lower() == "exit":
            print("Exiting...")
            break

        ranked_docs = process_query(
            query=query,
            inverted_index=inverted_index,
            vectorizer=vectorizer,
            tfidf_matrix=tfidf_matrix,
            bm25=bm25,
            documents=documents,
            default_operator="or"
        )

        if ranked_docs:
            print(f"\nFound {len(ranked_docs)} documents:")
            for doc_id, score in ranked_docs:
                real_index = doc_id - 1
                if 0 <= real_index < len(documents):
                    doc = documents[real_index]
                    title = doc.get('title', 'No Title')
                    original_def = doc.get('original_definition', 'No Definition')
                    snippet = original_def[:200] + "..." if len(original_def) > 200 else original_def
                    print(f"  Document ID: {doc_id}")
                    print(f"    Title: {title}")
                    print(f"    Definition: {snippet}")
                    print(f"    Score: {score:.4f}\n")
        else:
            print("No results found.")

if __name__ == "__main__":
    main()


### Επεξήγηση (Enhanced Search Engine)

- **`initialize_vsm(processed_texts)`**:
  - Χρησιμοποιεί `TfidfVectorizer` για να φτιάξει ένα **tfidf_matrix**.

- **`initialize_bm25(processed_texts)`**:
  - Χρησιμοποιεί `BM25Okapi` από τη βιβλιοθήκη `rank_bm25`.

- **`process_query(...)`**:
  - Ανιχνεύει αν το query περιέχει τους Boolean operators.
  - Αν είναι simple query, ρωτάει τον χρήστη ποιον αλγόριθμο κατάταξης θέλει:
    1. **TF-IDF Sum** (απλώς αθροίζει τα tf-idf scores)
    2. **Vector Space Model** (cosine similarity)
    3. **Okapi BM25**.
  - Επιστρέφει ταξινομημένη λίστα `(doc_id, score)`.

- **Στο τέλος** εμφανίζει τα έγγραφα και τη βαθμολογία (score).

## 7) Evaluation Script

**Σκοπός**: Εδώ ορίζουμε κάποια *test queries* και τους αντίστοιχους *relevant_docs*, τρέχουμε τη μηχανή αναζήτησης, και υπολογίζουμε μετρικές (precision, recall, F1).

In [ ]:
# Παράδειγμα evaluation
from search_engineV2 import (
    load_inverted_index,
    load_documents,
    preprocess_documents,
    initialize_vsm,
    initialize_bm25,
    process_query
)

def evaluate_system(test_queries, inverted_index, vectorizer, tfidf_matrix, bm25, documents):
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_queries = len(test_queries)

    for test_query in test_queries:
        query = test_query["query"]
        relevant_docs = set(test_query["relevant_docs"])

        # process_query επιστρέφει ranked_docs = [(doc_id, score), ...]
        ranked_docs = process_query(
            query=query,
            inverted_index=inverted_index,
            vectorizer=vectorizer,
            tfidf_matrix=tfidf_matrix,
            bm25=bm25,
            documents=documents,
            default_operator="or"
        )
        retrieved_docs = [doc_id for (doc_id, _) in ranked_docs]

        # Υπολογισμός precision, recall, f1
        retrieved_set = set(retrieved_docs)
        true_positives = len(retrieved_set & relevant_docs)

        precision = true_positives / len(retrieved_docs) if retrieved_docs else 0
        recall = true_positives / len(relevant_docs) if relevant_docs else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

        total_precision += precision
        total_recall += recall
        total_f1 += f1

        print(f"Query: {query}")
        print(f"  Relevant Docs: {relevant_docs}")
        print(f"  Retrieved Docs (top 5): {retrieved_docs[:5]}")
        print(f"  Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}\n")

    avg_precision = total_precision / num_queries
    avg_recall = total_recall / num_queries
    avg_f1 = total_f1 / num_queries
    print("=== AVERAGE RESULTS ===")
    print(f"Average Precision: {avg_precision:.2f}")
    print(f"Average Recall:    {avg_recall:.2f}")
    print(f"Average F1:        {avg_f1:.2f}")

def main():
    index_file = "inverted_index_tfidf.json"  # Χρησιμοποιούμε το TF-IDF ευρετήριο
    docs_file = "wiki_definitions_cleaned.json"

    inverted_index = load_inverted_index(index_file)
    documents = load_documents(docs_file)
    if not inverted_index or not documents:
        return

    processed_texts = preprocess_documents(documents)
    vectorizer, tfidf_matrix = initialize_vsm(processed_texts)
    bm25 = initialize_bm25(processed_texts)

    # Παράδειγμα test queries
    test_queries = [
        {
            "query": "machine learning",
            "relevant_docs": [3, 4, 10, 17, 18, 19]  # Π.χ.
        },
        {
            "query": "neural network",
            "relevant_docs": [7, 8, 9, 25]  # Παράδειγμα
        }
        # μπορείς να προσθέσεις κι άλλα
    ]

    evaluate_system(test_queries, inverted_index, vectorizer, tfidf_matrix, bm25, documents)

if __name__ == "__main__":
    main()


### Επεξήγηση (Evaluation)

- **`evaluate_system(test_queries, inverted_index, ...)`**:
  - Για κάθε ερώτημα, καλεί `process_query(...)` ώστε να πάρει τα `(doc_id, score)`.
  - Συγκρίνει τα αποτελέσματα (retrieved_docs) με τα `relevant_docs`.
  - Υπολογίζει μετρικές:
    - **Precision** = `TP / retrieved_docs`
    - **Recall** = `TP / total_relevant`
    - **F1** = `2 * (precision * recall) / (precision + recall)`
  - Εμφανίζει τα **μέσα** αποτελέσματα (Average Precision, Recall, F1).

- **`main()`**:
  - Φορτώνει το `inverted_index_tfidf.json` και τα έγγραφα.
  - Ορίζει κάποια test queries με τα doc_ids που θεωρούνται σχετικά.
  - Καλεί τη συνάρτηση evaluate_system.
